# Create Dataloader

In [1]:
import os
import pandas as pd
from dotmap import DotMap

from utils.io import load_yaml
from utils.io import load_pickle, save_pickle
from utils.transforms import anyDictListToCompose
from lightning_data_modules import VideoDataModule

server = 'local'
server = DotMap(load_yaml('../config/servers.yml')[server])

# meta_df_name = 'cholec80_dummy_transforms.pkl'
meta_df_name = 'cholec80_retransforms.pkl'
meta_df = pd.read_pickle(os.path.join(server.config.location, meta_df_name))
meta_df.aug_transforms = None  # no transforms for evaluation

# load cholec80
prefix = server.database.location
clip_length_in_frames = 2
frames_between_clips = 1
frame_rate = 5
train_split = 0.5
batch_size = 1
num_workers = 8
random_state = 42

# None if not initialized
test_md_name = 'cholec80_homography_regression_discrepancy_test_md_frame_rate_{}.pkl'.format(frame_rate)
# test_md_name = 'cholec80_dummy_test_md_frame_rate_{}.pkl'.format(frame_rate)

try:
    test_md = load_pickle(os.path.join(server.config.location, test_md_name))
except:
    test_md = None

dm = VideoDataModule(
    meta_df,
    prefix=prefix,
    clip_length_in_frames=clip_length_in_frames,
    frames_between_clips=frames_between_clips   ,
    frame_rate=frame_rate,
    train_split=train_split,
    batch_size=batch_size,
    num_workers=num_workers,
    random_state=random_state,
    test_metadata=test_md
)

_, _, test_md = dm.setup('test')

# store metadata
if test_md:
    save_pickle(os.path.join(server.config.location, test_md_name), test_md)

In [2]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [3]:
import cv2
import numpy as np
from tqdm import tqdm
import torch
from kornia import warp_perspective, tensor_to_image
import matplotlib.pyplot as plt

from lightning_modules import DeepImageHomographyEstimationModuleBackbone
from utils.processing import four_point_homography_to_matrix, image_edges
from utils.processing import FeatureHomographyEstimation
from utils.viz import yt_alpha_blend
from utils.io import generate_path

# feature-based model creationg stage
fd = cv2.SIFT_create()
fh = FeatureHomographyEstimation(fd)

# deep creationg stage
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_search/version_8'
# prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_search/version_7'
configs = load_yaml(os.path.join(prefix, 'config.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=99-step=44099.ckpt'), shape=configs['model']['shape'])
# model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=99-step=88099.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()

# data iterator
test_dl = dm.test_dataloader()

model._model.stem.conv.register_forward_hook(get_activation('stem.conv'))
model._model.s1.b1.f.c.register_forward_hook(get_activation('s1.b1.f.c'))
model._model.s1.b2.f.c.register_forward_hook(get_activation('s1.b2.f.c'))
model._model.s2.b1.f.c.register_forward_hook(get_activation('s2.b1.f.c'))
model._model.s2.b1.f.c.register_forward_hook(get_activation('s3.b1.f.c'))
model._model.s4.b2.f.c.register_forward_hook(get_activation('s4.b1.f.c'))
# model._model.s1.b1.se.f_ex[0].register_forward_hook(get_activation('s1.b1.se.f_ex.0'))
# model._model.s7.b1.se.f_ex[0].register_forward_hook(get_activation('s7.b1.se.f_ex.0'))
# model._model.s2.b1.f_ex[3].register_forward_hook(get_activation('s2.b1.f_ex.3'))
# model._model.s3.b1.f_ex[3].register_forward_hook(get_activation('s3.b1.f_ex.3'))
# model._model.s4.b1.f_ex[3].register_forward_hook(get_activation('s4.b1.f_ex.3'))

min_cnt = 119
cnt = 0
for batch in tqdm(test_dl):
    cnt += 1
    if cnt < min_cnt:
        continue
    # deep
    batch = dm.transfer_batch_to_device(batch, device)
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    img, wrp = vid[0,0].unsqueeze(0), vid[0,1].unsqueeze(0)
    duv = model(img, wrp)


    label = 's1.b2.f.c'
    # label = 's7.b1.se.f_ex.0'
    acts = activation[label].squeeze()
    # # fig, axarr = plt.subplots(act.size(0))
    # print(acts.shape)
    
    # # cv2.imshow('act', act[0].cpu().numpy())
    # # cv2.waitKey()
    
    # act = acts.mean(axis=0).cpu().numpy()
    # act = cv2.resize(act, (img.shape[-1], img.shape[-2]))
    # cv2.imshow(label, act)
    # cv2.waitKey()

    # for idx in range(acts.size(0)):
    #     print(idx)
    #     act = acts[idx].cpu().numpy()
    #     act = cv2.resize(act, (img.shape[-1], img.shape[-2]))
    #     mul = np.multiply(act, img.cpu().numpy().squeeze())
    #     mul = np.transpose(mul, (1, 2, 0))
    #     mul = mul[...,::-1]
    #     cv2.imshow(label, mul)
    #     act = (act-act.min())/(act.max()-act.min())
    #     cv2.imshow('act', act)
    #     cv2.imshow('img', tensor_to_image(img.cpu())[...,::-1])
    #     cv2.waitKey()
    
    idx1 = 33 # bg
    idx2 = 17 # fb
    idx3 = 25 # fg 25
    idx4 = 6


    act1 = acts[idx1].cpu().numpy()
    act1 = cv2.resize(act1, (img.shape[-1], img.shape[-2]))
    mul1 = np.multiply(act1, img.cpu().numpy().squeeze())
    mul1 = np.transpose(mul1, (1, 2, 0))
    mul1 = mul1[...,::-1]

    act2 = acts[idx2].cpu().numpy()
    act2 = cv2.resize(act2, (img.shape[-1], img.shape[-2]))
    mul2 = np.multiply(act2, img.cpu().numpy().squeeze())
    mul2 = np.transpose(mul2, (1, 2, 0))
    mul2 = mul2[...,::-1]

    act3 = acts[idx3].cpu().numpy()
    act3 = cv2.resize(act3, (img.shape[-1], img.shape[-2]))
    mul3 = np.multiply(act3, img.cpu().numpy().squeeze())
    mul3 = np.transpose(mul3, (1, 2, 0))
    mul3 = mul3[...,::-1]

    act4 = acts[idx4].cpu().numpy()
    act4 = cv2.resize(act4, (img.shape[-1], img.shape[-2]))
    mul4 = np.multiply(act4, img.cpu().numpy().squeeze())
    mul4 = np.transpose(mul4, (1, 2, 0))
    mul4 = mul4[...,::-1]

    cv2.imshow('mul1', mul1)
    cv2.imshow('mul2', mul2)
    cv2.imshow('mul3', mul3)
    act4 = (act4-act4.min())/(act4.max()-act4.min())
    cv2.imshow('act4', act4)
    cv2.imshow('img', tensor_to_image(img.cpu())[...,::-1])
    cv2.waitKey()

Running with CUDA backend.
  0%|          | 0/89715 [00:00<?, ?it/s]/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "


In [2]:
import cv2
import numpy as np
from tqdm import tqdm
import torch
from kornia import warp_perspective, tensor_to_image
import matplotlib.pyplot as plt

from lightning_modules import DeepImageHomographyEstimationModuleBackbone
from utils.processing import four_point_homography_to_matrix, image_edges
from utils.processing import FeatureHomographyEstimation
from utils.viz import yt_alpha_blend
from utils.io import generate_path

# feature-based model creationg stage
fd = cv2.SIFT_create()
fh = FeatureHomographyEstimation(fd)

# deep creationg stage
prefix = '/home/martin/Tresors/homography_imitation_learning_logs/deep_image_homography_estimation_backbone_search/version_8'
configs = load_yaml(os.path.join(prefix, 'config.yml'))
model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(prefix, 'checkpoints/epoch=99-step=44099.ckpt'), shape=configs['model']['shape'])

device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

model.to(device)
model = model.eval()
model.freeze()

# data iterator
test_dl = dm.test_dataloader()

batch = next(iter(test_dl))
batch = dm.transfer_batch_to_device(batch, device)
vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

ones = torch.ones(vid.shape, device=vid.device, requires_grad=True)

img, wrp = vid[0,0].unsqueeze(0), vid[0,1].unsqueeze(0)
duv_target = model(img, wrp).detach()


epochs = 100

opt = torch.optim.Adam([ones])
loss = torch.nn.MSELoss()

for epoch in range(epochs):
    print(epoch)
    opt.zero_grad()
    vid *= ones

    img, wrp = vid[0,0].unsqueeze(0), vid[0,1].unsqueeze(0)
    duv = model(img, wrp)

    l = loss(duv, duv_target)
    # l.backward()
    # opt.step()







Running with CUDA backend.
/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
